In [ ]:
# Teste

In [ ]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/LeticiaSouza04/Bootcamp/main/UNdata_Export_20240412_202657352.csv"


In [ ]:
dados_un = pd.read_csv(url_commodities)
dados_un

,Country or Area,Year,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity
0,Afghanistan,2019,All Commodities,Export,8.704885e+08,NaN,No Quantity,NaN
1,Afghanistan,2019,All Commodities,Import,8.568014e+09,NaN,No Quantity,NaN
2,Afghanistan,2019,All Commodities,Re-Export,6.655197e+06,NaN,No Quantity,NaN
3,Afghanistan,2018,All Commodities,Import,7.406590e+09,NaN,No Quantity,NaN
4,Afghanistan,2018,All Commodities,Re-Export,9.263097e+06,NaN,No Quantity,NaN
...,...,...,...,...,...,...,...,...
11175,Zimbabwe,2000,All Commodities,Re-Export,3.202940e+07,NaN,No Quantity,NaN
11176,Zimbabwe,2000,All Commodities,Export,1.607053e+09,NaN,No Quantity,NaN
11177,Zimbabwe,2000,All Commodities,Import,2.016521e+09,NaN,No Quantity,NaN
11178,Zimbabwe,1995,All Commodities,Import,2.658853e+09,NaN,No Quantity,NaN
